In [20]:
import os
import sys
sys.path.insert(0, '/share/project/yfl/codebase/stable_diffusion_2.0/stablediffusion/')
from lm.dcn_clip import DCNCLIP30M1024
from ldm.modules.encoders.modules import FrozenOpenCLIPEmbedder

os.environ["CUDA_VESIBLE_DEVICES"] = '7'

altclip = DCNCLIP30M1024().to("cuda")
openclip = FrozenOpenCLIPEmbedder().to("cuda")

^_^ Using the right lm model!!!
Language model Loaded!!!^_^


In [18]:
## sample出10K的prompts

import json
import random
import shutil

n = 1000

#### samle 10000个图文对
path = '/share/project/yfl/codebase/nb/laion6plus/part_1.json'
with open(path, 'r') as fn:
    d = json.load(fn)


samples = random.sample(d, n)

texts = []
for sample in samples:
    texts.append(sample['source_caption'])

In [18]:
import torch
from tqdm import tqdm
from tqdm.contrib import tzip

def compared_altwithopen(texts_en, texts_other=None):
    loss_func  = torch.nn.MSELoss()
    loss_result_min = []

    if texts_other:
        for text_en, text_other in tzip(texts_en, texts_other, desc="Evaluating:"):
        
            loss_result = []
            emb1 = altclip.encode(text_other)
            emb2 = openclip.encode(text_en)

            for token in emb2[0]:
                cls = emb1[0][0].to("cuda")
                loss_result.append(loss_func(token, cls))
            loss_result_min.append(min(loss_result))
    else:
        for text in tqdm(texts_en, desc="Evaluating:"):
            
            loss_result = []
            emb1 = altclip.encode(text)
            emb2 = openclip.encode(text)

            for token in emb2[0]:
                cls = emb1[0][0].to("cuda")
                loss_result.append(loss_func(token, cls))
            loss_result_min.append(min(loss_result))
             
    print("Average MSE: {}".format(sum(loss_result_min)/len(loss_result_min)))

def compared_altwithalt(texts1, texts2):
    loss_func  = torch.nn.MSELoss()
    loss_result = []

    for text1, text2 in tqdm(zip(texts1, texts2), desc="Evaluating:"):
        
        emb1 = altclip.encode(text1)
        emb2 = altclip.encode(text2)

        cls1 = emb1[0][0].to("cuda")
        cls2 = emb2[0][0].to("cuda")
        loss_result.append(loss_func(cls1, cls2))
        
    print("Average MSE: {}".format(sum(loss_result)/len(loss_result)))

In [5]:
import json

with open("./text_embedding_analysis/texts_dict.json", 'r') as fn:
    d1 = json.load(fn)

with open("./text_embedding_analysis/texts_dict_ar_AR.json", 'r') as fn:
    d2 = json.load(fn)

In [14]:
texts1 = []
texts2 = []
for it in d1:
    texts1.append(it["source_caption"])
for it in d2:
    texts2.append(it["target_caption"])

In [21]:
compared_altwithopen(texts_en=texts1, texts_other=texts2)

Evaluating::   0%|          | 0/1000 [00:00<?, ?it/s]

Average MSE: 0.33988305926322937


# Misc

In [3]:
import json
with open('/share/project/yfl/codebase/stable_diffusion_2.0/stablediffusion/nb/validation/gt.json', 'r') as fn:
    data = json.load(fn)

exp_name = 'sd2.1'
# 存储对应的json文件
result = []
for datum in data:
    caption = datum['source_caption']
    original_dir = datum['image']
    file_name = original_dir.split("/")[-1]
    result.append(
        {
            "caption": caption,
            "image": '/share/project/yfl/codebase/stable_diffusion_2.0/stablediffusion/nb/validation/generate_imgs/' + exp_name + '/' + file_name,
        }
    )

with open('./validation/' + exp_name + '.json', 'w') as fn:
    json.dump(result, fn)